In [3]:
#!pip install twython

In [20]:
from twython import Twython
import pandas as pd
import numpy as np
import datetime as dt
import time
import yaml
import dask.dataframe as dd
from dask.multiprocessing import get

In [5]:
APP_KEY = 'OumzA854PAnWwA8VNjwZ6O1T1'
APP_SECRET = "aHWN7hVHFYGTJIqLG6d1sUgT2tzelmB7LNxtQjJz0RkmLDIFGW"

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [6]:
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [7]:
locationsTemp=["chennai","delhi","india","ahmedabad","gujarat","tamil","jammu","srinagar","imphal","lucknow",
                "bengaluru","bangalore","bathinda","chandigarh","ludhiana","amritsar","punjab","arunachal","assam",
                "kolkata","westbengal","kerala","karnataka","andhra","telangana","hyderabad","madhya","uttar",
                "maharashtra","haryana",'Jammu','Srinagar','Delhi-NewDelhi','Bathinda','Dehradun','Chandigarh',
                'Ludhiana','Amritsar','Imphal','Lucknow','Jalandhar','Kolkata','Guwahati','Chennai','Patiala',
                'Bhubaneswar','Bengaluru','Patna','Jaipur','Coimbatore','Hyderabad','Ranchi','Thiruvananthapuram',
                'Shimla','Sangrur','Ahmedabad','Karnal','Pulwama','Puducherry','Gurgaon','Agartala','Madurai',
                'Tiruchirappalli','Bangkok','Salem']

locations = set()
for location in locationsTemp:
    locations.add(location.lower())
locations = pd.DataFrame(list(locations))
locations.columns = ["Locations"]
locations.to_csv("Locations of Interest.csv")

In [8]:
keywords = ['protest','protests','riot','riots','violence','unrest','clash','bandh','issue','rally','demonstration','election','polls','attac']
keywords = pd.DataFrame(keywords)
keywords.columns = ["Keywords"]
keywords.to_csv("Keywords.csv")

In [9]:
locations = pd.read_csv("Locations of Interest.csv", index_col=0)
keywords = pd.read_csv("Keywords.csv", index_col=0)

In [10]:
keywordsQuery = ''.join(map(str, [keyword+" OR " for keyword in list(keywords['Keywords'])]))[:-4]

In [11]:
start = 0
to_return = []
loc_list =locations["Locations"].values.tolist()
len_loc = len(loc_list) // 5 + 1
for i in range(5):
    to_return.append(loc_list[start:start+len_loc])
    start+=len_loc
    
locationQueries=[]
for row in to_return:
    locationQueries.append(''.join(map(str, [location+" OR " for location in row]))[:-4])
    

In [9]:
print("Locations Queries\n\n", locationQueries,"\n")
print("Keywords Query\n\n", keywordsQuery)

Locations Queries

 ['patiala OR tamil OR imphal OR maharashtra OR chennai OR madurai OR haryana OR srinagar OR delhi-newdelhi OR sangrur OR uttar', 'bangalore OR patna OR india OR salem OR agartala OR thiruvananthapuram OR tiruchirappalli OR coimbatore OR jalandhar OR telangana OR puducherry', 'shimla OR karnal OR ahmedabad OR ranchi OR bangkok OR assam OR chandigarh OR arunachal OR amritsar OR kolkata OR dehradun', 'jammu OR kerala OR lucknow OR punjab OR hyderabad OR delhi OR bengaluru OR madhya OR pulwama OR ludhiana OR jaipur', 'guwahati OR gujarat OR andhra OR gurgaon OR karnataka OR bhubaneswar OR westbengal OR bathinda'] 

Keywords Query

 protest OR protests OR riot OR riots OR violence OR unrest OR clash OR bandh OR issue OR rally OR demonstration OR election OR polls OR attac


In [10]:
date = dt.datetime.now()
cnt = 0
tweetList = []
for locationQuery in locationQueries:
    tweets = twitter.cursor(twitter.search, q=keywordsQuery + ' (' +locationQuery + ') -filter:retweets', count=100, tweet_mode = "extended")
    for item in tweets:
        if (cnt%10000 == 0):
            print("Progress: "+str(cnt))
        tweetList.append(item)
        cnt+=1
        if cnt > 40000:
            print("Thread sleep")
            time.sleep(60*15+5)
            cnt = 0
tweetsDF = pd.DataFrame(tweetList)
tweetsDF.to_csv("tweets_collected_"+str(date.day)+str(date.strftime("%m"))+".csv")

Progress: 0
Progress: 10000


C:\Users\Dan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: generator 'Twython.cursor' raised StopIteration
  


Progress: 20000
Progress: 30000
Progress: 40000
Thread sleep
Progress: 0
Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Thread sleep
Progress: 0
Progress: 10000


In [89]:
tweetsDF1 = pd.read_csv("Tweets3Weeks1304_1.csv", dtype={'id':object})
tweetsDF2 = pd.read_csv("Tweets3Weeks1304_2.csv", dtype={'id':object})
tweetsDF3 = pd.read_csv("Tweets3Weeks1304_3.csv", dtype={'id':object})
tweetsDF4 = pd.read_csv("Tweets3Weeks1304_4.csv", dtype={'id':object})
tweetsDF5 = pd.read_csv("Tweets3Weeks1304_5.csv", dtype={'id':object})
tweetsDF6 = pd.read_csv("Tweets3Weeks1304_6.csv", dtype={'id':object})

C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
tweetsDF = pd.concat([tweetsDF1,tweetsDF2,tweetsDF3,tweetsDF4,tweetsDF5,tweetsDF6])

In [88]:
tweetsDF.to_csv("Tweets3Weeks_2.csv")

In [100]:
tweetsDF1 = tweetsDfs[0]
tweetsDF2 = tweetsDfs[1]
tweetsDF3 = tweetsDfs[2]
tweetsDF4 = tweetsDfs[3]
tweetsDF5 = tweetsDfs[4]
tweetsDF6 = tweetsDfs[5]

In [12]:
pwd

'/home/aashish_jain/Repos/Election-Violence-Prediction/twitter'

In [14]:
tweetsDF = pd.read_pickle("../Tweets3Weeks_part2.pkl")

In [71]:
# from tqdm import tqdm_notebook as tqdm
def get_features(row):
    row["userlocation"] = row["user"]["location"]
    row["username"] = row["user"]["name"]
    row["userscreen_name"] = row["user"]["screen_name"]
    row["userdescription"] = row["user"]["description"]
    row["userfollowers_count"] = row["user"]["followers_count"]
    row["userfriends_count"] = row["user"]["friends_count"]
    row["userlisted_count"] = row["user"]["listed_count"]
    row["userfavourites_count"] = row["user"]["favourites_count"]
    row["userverified"] = row["user"]["verified"]
    row["userstatuses_count"] = row["user"]["statuses_count"]
    row["userfollowing"] = row["user"]["following"]
    row["userfollow_request_sent"] = row["user"]["follow_request_sent"]
    row["usercontributors_enabled"] = row["user"]["contributors_enabled"]
    sepr =  "" 
    l = []
    for location in locations['Locations']: 
        if (location in str(row["full_text"]).lower()):
            l.append(location)
    row["extracted_location"] = ','.join(l)
    return row

In [73]:
processed_data = tweetsDF.apply(get_features, axis=1)

In [86]:
locations['Locations']
acledLocations = ['Jammu',
 'Srinagar',
 'Delhi-New Delhi',
 'Bathinda',
 'Dehradun',
 'Chandigarh',
 'Ludhiana',
 'Amritsar',
 'Imphal',
 'Lucknow',
 'Jalandhar',
 'Kolkata',
 'Guwahati',
 'Chennai',
 'Patiala',
 'Bhubaneswar',
 'Bengaluru',
 'Patna',
 'Jaipur',
 'Coimbatore',
 'Hyderabad',
 'Ranchi',
 'Thiruvananthapuram',
 'Shimla',
 'Sangrur',
 'Ahmedabad',
 'Karnal',
 'Pulwama',
 'Puducherry',
 'Gurgaon',
 'Agartala',
 'Madurai',
 'Tiruchirappalli',
 'Bangkok',
 'Salem','India']

In [217]:
def generate_locations(row):
    dfrow_list = []
    for location in acledLocations: 
        tempLocation = location
        tempLocation2 = location
        if (tempLocation == "Delhi-New Delhi"):
            tempLocation = "delhi"
        elif (tempLocation == "Bengaluru"):
            tempLocation = "bangalore"
            tempLocation2 = "karnataka"
        elif (tempLocation == "Kolkata"):
            tempLocation = "west bengal"
        elif (tempLocation == "Guwahati"):
            tempLocation = "assam"
        elif (tempLocation in ("Lucknow", "Dehradun")):
            tempLocation = "uttar"
        elif (tempLocation in ("Sangrur", "Ludhiana", "Amritsar", "Chandigarh", "Bathinda", "Jalandhar", "Patiala")):
            tempLocation = "punjab"
        elif (tempLocation in ("Karnal", "Gurgaon", "Chandigarh")):
            tempLocation2 = "haryana"
        elif (tempLocation == "Pulwama"):
            tempLocation2 = "jammu"
        elif (tempLocation == "Ahmedabad"):
            tempLocation = "gujarat"
        elif (tempLocation in ("Chennai", "Coimbatore", "Madurai", "Tiruchirappalli", "Salem")):
            tempLocation = "tamil"
        elif (tempLocation in ("Imphal", "Agartala")):
            tempLocation = "arunachal"
        elif (tempLocation == "Hyderabad"):
            tempLocation = "andhra"
            tempLocation2 = "telangana"
        elif (tempLocation == "Thiruvananthapuram"):
            tempLocation = "kerala"
        if (tempLocation.lower() in row["extracted_location"] or location.lower() in row["extracted_location"] or tempLocation2 in row["extracted_location"]):
            row["finallocation"] = location
            dfrow_list.append(row.copy())
#     print(row)
    return pd.DataFrame(dfrow_list)

## Extra Stuff

In [221]:
posssed_df = pd.concat(processed_data.apply(generate_locations, axis = 1).tolist())

In [220]:
len(posssed_df)

1161

In [165]:
posssed_df.to_pickle("Tweets3Weeks_part2_final.pkl")

In [186]:
len(processed_data[processed_data["extracted_location"] == ""])

58733

In [201]:
processed_data[1000:2000].apply(generate_locations, axis = 1)

1000    Empty DataFrame
Columns: []
Index: []
1001    Empty DataFrame
Columns: []
Index: []
1002    Empty DataFrame
Columns: []
Index: []
1003    Empty DataFrame
Columns: []
Index: []
1004    Empty DataFrame
Columns: []
Index: []
1005    Empty DataFrame
Columns: []
Index: []
1006    Empty DataFrame
Columns: []
Index: []
1007    Empty DataFrame
Columns: []
Index: []
1008    Empty DataFrame
Columns: []
Index: []
1009    Empty DataFrame
Columns: []
Index: []
1010    Empty DataFrame
Columns: []
Index: []
1011    Empty DataFrame
Columns: []
Index: []
1012    Empty DataFrame
Columns: []
Index: []
1013    Empty DataFrame
Columns: []
Index: []
1014    Empty DataFrame
Columns: []
Index: []
1015    Empty DataFrame
Columns: []
Index: []
1016    Empty DataFrame
Columns: []
Index: []
1017    Empty DataFrame
Columns: []
Index: []
1018    Empty DataFrame
Columns: []
Index: []
1019    Empty DataFrame
Columns: []
Index: []
1020    Empty DataFrame
Columns: []
Index: []
1021    Empty DataFrame
Columns: [

In [202]:
processed_data[1000:2000]["extracted_location"]

1000                     punjab,karnataka
1001                                     
1002                              gujarat
1003                                     
1004                              gujarat
1005            karnataka,haryana,gujarat
1006                              gujarat
1007                              gujarat
1008                              gujarat
1009                               ranchi
1010              andhra,karnataka,kerala
1011                              gujarat
1012                              gujarat
1013                              bangkok
1014                              gujarat
1015                                jammu
1016                                jammu
1017                                     
1018                              gujarat
1019                            karnataka
1020                            karnataka
1021                            karnataka
1022                              gujarat
1023                              

In [200]:
posssed_df

""


In [4]:
tweets1 = pd.read_csv("tweets_collected_304.csv", dtype={'id': object})
tweets2 = pd.read_csv("tweets_collected_1104.csv", dtype={'id': object} , lineterminator='\n')
tweets3 = pd.read_csv("tweets_collected_2703.csv", dtype={'id': object})

C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,4,8,9,18,27,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,9,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(tweets1.shape)
print(tweets2.shape)
print(tweets3.shape)
tweets = tweets1.append(tweets2.append(tweets3))

C:\Users\Dan\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


(78313, 33)
(92273, 33)
(68253, 33)


In [6]:
tweets.shape

(238839, 34)

In [7]:
cnt = 1
rowInd = 1
tempList = []
tweetList = []
tweetsDfs = []
date = dt.datetime.now()
for index, row in tweets.iterrows():
    tempList.append(row["id"])
    if cnt%100 == 0:
        try:
            tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
            tempList = []
        except:
            print("Succesful till: "+cnt)
    if (cnt%10000 == 0):
        print("Progress: "+str(cnt))
        if cnt >= 40000:
            tweetsDfs.append(pd.DataFrame(tweetList))
#             tweetsDf.to_csv("Tweets3Weeks"+str(date.day)+str(date.strftime("%m"))+"_"+str(rowInd)+".csv")
            tweetList = []
            rowInd+=1
#             print("Thread sleep")
#             time.sleep(60*15+5)
            cnt = 0
    cnt+=1
tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
tweetsDfs.append(pd.DataFrame(tweetList))

Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Progress: 10000
Progress: 20000
Progress: 30000


In [11]:
tweetsDF = pd.concat(tweetsDfs)

In [13]:
tweetsDF.index = pd.np.arange(len(tweetsDF))

In [23]:
tweetsDF = pd.read_pickle("Tweets3Weeks_part2.pkl")

In [24]:
tweetsDF

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,source,truncated,user,withheld_in_countries
0,None,None,Wed Apr 03 04:24:07 +0000 2019,"[16, 215]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,"@AudreyTruschke I THINK HE LOST HID MEMORY,, H...",None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 294672296, 'id_str': '294672296', 'name...",NaN
1,None,None,Wed Apr 03 05:17:12 +0000 2019,"[0, 224]","{'hashtags': [{'text': 'ElectionCommission', '...","{'media': [{'id': 1113309107711238146, 'id_str...",1,False,Pro-democracy activist Siriwith Serithiwat col...,None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 1326375108, 'id_str': '1326375108', 'na...",NaN
2,None,None,Wed Apr 03 05:02:38 +0000 2019,"[0, 201]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,3,False,"Soon, narendramodi will address a massive rall...",None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",False,"{'id': 2989893645, 'id_str': '2989893645', 'na...",NaN
3,None,None,Wed Apr 03 05:04:36 +0000 2019,"[0, 100]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1113306122855702529, 'id_str...",51,False,Prime Minister Narendra Modi to address a publ...,None,...,NaN,NaN,NaN,NaN,8,False,"<a href=""https://about.twitter.com/products/tw...",False,"{'id': 36327407, 'id_str': '36327407', 'name':...",NaN
4,None,None,Wed Apr 03 04:15:34 +0000 2019,"[0, 121]","{'hashtags': [{'text': 'Bengaluru', 'indices':...",NaN,16,False,"ತೊಟ್ಟಿಲು ತೂಗಿ, ಸಿಎಂ ಪದವಿಗೆ ಮೊರೆ ಹೋದ್ರಾ ಡಿಕೆಶಿ?...",None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 471911020, 'id_str': '471911020', 'name...",NaN
5,None,None,Wed Apr 03 04:12:45 +0000 2019,"[0, 187]","{'hashtags': [{'text': 'May23WithArnab', 'indi...",NaN,43,False,Shocking remark by former J&amp;K CM Farooq Ab...,None,...,NaN,NaN,NaN,NaN,19,False,"<a href=""https://about.twitter.com/products/tw...",False,"{'id': 811972460560019456, 'id_str': '81197246...",NaN
6,None,None,Wed Apr 03 04:44:37 +0000 2019,"[0, 185]","{'hashtags': [{'text': 'ModiAgain2019', 'indic...",NaN,2,False,Massive turnout in Shri @AmitShah yesterday's ...,None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://www.facebook.com/twitter"" rel=...",False,"{'id': 420358585, 'id_str': '420358585', 'name...",NaN
7,None,None,Wed Apr 03 05:15:40 +0000 2019,"[0, 207]",{'hashtags': [{'text': 'LokSabhaElections2019'...,"{'media': [{'id': 1113308916803178497, 'id_str...",5,False,#LokSabhaElections2019 | Prime Minister Naren...,None,...,NaN,NaN,NaN,NaN,1,False,"<a href=""https://about.twitter.com/products/tw...",False,"{'id': 284920800, 'id_str': '284920800', 'name...",NaN
8,None,None,Wed Apr 03 04:09:21 +0000 2019,"[0, 101]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,"चुनावी घोषणा पत्र से राष्ट्रवाद, कश्मीर विषय प...",None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 46853675, 'id_str': '46853675', 'name':...",NaN
9,None,None,Wed Apr 03 05:15:50 +0000 2019,"[0, 279]","{'hashtags': [{'text': 'CongressManifesto', 'i...",NaN,5,False,When the BJP talks about doing away with AFSPA...,None,...,NaN,NaN,NaN,NaN,2,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 836986783699103744, 'id_str': '83698678...",NaN


In [95]:
tweetsDfs.append(pd.DataFrame(tweetList))

In [99]:
tweetsDfs[5]

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,source,truncated,user,withheld_in_countries
0,None,None,Mon Mar 25 04:40:41 +0000 2019,"[0, 146]",{'hashtags': [{'text': 'LokSabhaElections2019'...,NaN,2,False,Lok Sabha 2019 Election guide | \nHere is the ...,None,...,NaN,NaN,NaN,NaN,2,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 403670899, 'id_str': '403670899', 'name...",NaN
1,None,None,Mon Mar 25 04:52:45 +0000 2019,"[0, 268]","{'hashtags': [{'text': 'Goriyas', 'indices': [...",NaN,11,False,"Assamese Muslims, who are broadly known as #Go...",None,...,NaN,NaN,NaN,NaN,4,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 256495314, 'id_str': '256495314', 'name...",NaN
2,None,None,Mon Mar 25 04:44:06 +0000 2019,"[0, 112]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1110039630013960194, 'id_str...",2,False,Lok Sabha election 2019: Congress fields Karti...,None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""https://www.hootsuite.com"" rel=""nofol...",False,"{'id': 161318053, 'id_str': '161318053', 'name...",NaN
3,None,None,Mon Mar 25 04:35:47 +0000 2019,"[0, 169]","{'hashtags': [{'text': 'Gurugram', 'indices': ...",NaN,2,False,"INDIA: 🇮🇳\n\nThe Quint: Quick, Alert &amp; Bra...",None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 867053964310794241, 'id_str': '86705396...",NaN
4,None,None,Mon Mar 25 04:30:27 +0000 2019,"[67, 337]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1110036190957166592, 'id_str...",0,False,@sorabjain @NeonKashmir @zainabbasqamar7 @Praj...,None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 744128402878193664, 'id_str': '74412840...",NaN
5,None,None,Mon Mar 25 04:47:06 +0000 2019,"[0, 83]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,2,False,दिग्विजय पर तंज कसते हुए शिवराज सिंह बोले- बंट...,None,...,NaN,NaN,NaN,NaN,1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 728173094666272768, 'id_str': '72817309...",NaN
6,None,None,Mon Mar 25 04:42:34 +0000 2019,"[51, 210]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,@zainy80 @majorgauravarya @BBCUrdu @OfficialDG...,None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1109856684497948673, 'id_str': '1109856...",NaN
7,None,None,Mon Mar 25 04:51:20 +0000 2019,"[0, 136]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,3,False,Lok Sabha polls 2019: 12 battles that will dec...,None,...,NaN,NaN,NaN,NaN,2,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 38624666, 'id_str': '38624666', 'name':...",NaN
8,None,None,Mon Mar 25 04:46:06 +0000 2019,"[0, 300]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,7,False,"Must read: In Yogi's UP, the law &amp; the pol...",None,...,NaN,NaN,NaN,NaN,3,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 781470578, 'id_str': '781470578', 'name...",NaN
9,None,None,Mon Mar 25 04:52:33 +0000 2019,"[28, 293]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,@Airtel_Presence @DoT_India Why are you expect...,None,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 345478377, 'id_str': '345478377', 'name...",NaN


In [23]:
tweetsDf = pd.DataFrame(tweetList)
tweetsDf.to_csv("Tweets3Weeks"+str(date.day)+str(date.strftime("%m"))+"_"+str(rowInd)+".csv")